In [46]:
from selenium import webdriver
import pandas as pd
import numpy as np
import time

In [47]:
import os
from openpyxl import load_workbook

#https://stackoverflow.com/questions/38074678/append-existing-excel-sheet-with-new-dataframe-using-python-pandas

def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,truncate_sheet=False, **to_excel_kwargs):
    # Excel file doesn't exist - saving and exiting
    if not os.path.isfile(filename):
        df.to_excel(
            filename,
            sheet_name=sheet_name, 
            startrow=startrow if startrow is not None else 0, 
            **to_excel_kwargs)
        return
    
    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl', mode='a')

    # try to open an existing workbook
    writer.book = load_workbook(filename)
    
    # get the last row in the existing Excel sheet
    # if it was not specified explicitly
    if startrow is None and sheet_name in writer.book.sheetnames:
        startrow = writer.book[sheet_name].max_row

    # truncate sheet
    if truncate_sheet and sheet_name in writer.book.sheetnames:
        # index of [sheet_name] sheet
        idx = writer.book.sheetnames.index(sheet_name)
        # remove [sheet_name]
        writer.book.remove(writer.book.worksheets[idx])
        # create an empty sheet [sheet_name] using old index
        writer.book.create_sheet(sheet_name, idx)
    
    # copy existing sheets
    writer.sheets = {ws.title:ws for ws in writer.book.worksheets}

    if startrow is None:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, **to_excel_kwargs)

    # save the workbook
    writer.save()

In [48]:
import requests

while(True):
    headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15","Accept-Language": "en-gb","Accept-Encoding":"br, gzip, deflate","Accept":"test/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Referer":"http://www.google.com/"}
    url = "https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY"
    page = requests.get(url,headers=headers)
    
    if page.ok:
        break



In [49]:
from datetime import datetime
d = datetime.now().time()
ctime = d.strftime('%H:%M')
ctime

'21:48'

In [50]:
nifty_margin = 500
bank_nifty_margin = 1000
stock_margin = 10

In [12]:
col_list = ['strikePrice','expiryDate','openInterest']
data_format = ['Date','Stock/Index','Expiry','Right','Current OI','Previous Minute OI','OI Change',
               '% Up/Down from Last Minute OI','Right','Current OI','Previous Minute OI','OI Change',
               '% Up/Down from Last Minute OI']
base_df = pd.DataFrame(columns = data_format)
# append_df_to_excel("master.xlsx",base_df,sheet_name=str(i), index=False)

NameError: name 'i' is not defined

In [13]:
selected_margin = nifty_margin

In [14]:
import json
res = json.loads(page.text)['records']['data']

In [15]:
underlying_value = int(res[0]['PE']['underlyingValue'])
type(underlying_value)

int

In [16]:
# date = input("Enter date in format yyyy-mm-dd ")
date = '2021-05-27'

In [53]:
data = [d.get('CE') for d in res]
data_cleaned = list(filter(None, data))
df_call = pd.DataFrame(data_cleaned)

data = [d.get('PE') for d in res]
data_cleaned = list(filter(None, data))
df_put = pd.DataFrame(data_cleaned)

df_put['expiryDate'] = pd.to_datetime(df_put['expiryDate'])
df_call['expiryDate'] = pd.to_datetime(df_call['expiryDate'])


df_put.sort_values(by=['expiryDate','strikePrice'],inplace=True)
df_call.sort_values(by=['expiryDate','strikePrice'],inplace=True)

df_put = df_put[col_list]
df_call = df_call[col_list]

In [54]:
df_put = df_put[df_put['expiryDate'] == date]
df_call = df_call[df_call['expiryDate'] == date]
# df_put 



df_put['strikePrice'] = df_put['strikePrice'].map(int)
df_call['strikePrice'] = df_call['strikePrice'].map(int)

In [56]:
# print(res)

In [57]:
# min, max = ((100-percentage)*0.01)*(df['underlyingValue'][0]), ((100+percentage)*0.01)*(df['underlyingValue'][0])
# df_percent = df.loc[(df['strikePrice']>=min) & (df['strikePrice']<=max)]
# df_percent

In [58]:
df_put = df_put[(df_put['strikePrice'] >= underlying_value-selected_margin) & (df_put['strikePrice']<=underlying_value+selected_margin)]
df_call = df_call[(df_call['strikePrice'] >= underlying_value-selected_margin) & (df_call['strikePrice']<=underlying_value+selected_margin)]
df_put['RIGHT'] = 'PUT'
df_call['RIGHT'] = 'CALL'

In [59]:
max_sp = df_call.strikePrice.max
min_sp = df_call.strikePrice.min

In [60]:
sp_range = df_put.strikePrice.unique().tolist()

In [61]:
for i in sp_range:
    temp_df_put = df_put[df_put['strikePrice']==i]
    temp_df_call = df_call[df_call['strikePrice']==i]

In [62]:
temp1 = df_call
temp2 = df_put
# Ispe bharosa mat karna
temp2.merge(temp1,left_index=True,right_index=True,on='strikePrice')

,strikePrice,expiryDate_x,openInterest_x,RIGHT_x,expiryDate_y,openInterest_y,RIGHT_y
426,14700,2021-05-27,31764,PUT,2021-05-27,22670,CALL
436,14750,2021-05-27,7128,PUT,2021-05-27,1022,CALL
457,14850,2021-05-27,7821,PUT,2021-05-27,2153,CALL
548,15300,2021-05-27,5082,PUT,2021-05-27,33110,CALL


# Starting

In [63]:
# date = input("Enter date in format yyyy-mm-dd ")


In [64]:
df_put

,strikePrice,expiryDate,openInterest,RIGHT
426,14700,2021-05-27,31764,PUT
436,14750,2021-05-27,7128,PUT
447,14800,2021-05-27,40094,PUT
457,14850,2021-05-27,7821,PUT
467,14900,2021-05-27,39429,PUT
478,14950,2021-05-27,8008,PUT
486,15000,2021-05-27,61174,PUT
501,15050,2021-05-27,9612,PUT
511,15100,2021-05-27,35576,PUT
513,15150,2021-05-27,9062,PUT


In [65]:
df_call

,strikePrice,expiryDate,openInterest,RIGHT
405,14700,2021-05-27,10115,CALL
415,14750,2021-05-27,653,CALL
426,14800,2021-05-27,22670,CALL
436,14850,2021-05-27,1022,CALL
446,14900,2021-05-27,13897,CALL
457,14950,2021-05-27,2153,CALL
465,15000,2021-05-27,35065,CALL
480,15050,2021-05-27,7919,CALL
490,15100,2021-05-27,30657,CALL
492,15150,2021-05-27,6988,CALL


In [66]:
df = df_put.merge(df_call, on='strikePrice')
# df['Time'] = ctime
df

,strikePrice,expiryDate_x,openInterest_x,RIGHT_x,expiryDate_y,openInterest_y,RIGHT_y
0,14700,2021-05-27,31764,PUT,2021-05-27,10115,CALL
1,14750,2021-05-27,7128,PUT,2021-05-27,653,CALL
2,14800,2021-05-27,40094,PUT,2021-05-27,22670,CALL
3,14850,2021-05-27,7821,PUT,2021-05-27,1022,CALL
4,14900,2021-05-27,39429,PUT,2021-05-27,13897,CALL
5,14950,2021-05-27,8008,PUT,2021-05-27,2153,CALL
6,15000,2021-05-27,61174,PUT,2021-05-27,35065,CALL
7,15050,2021-05-27,9612,PUT,2021-05-27,7919,CALL
8,15100,2021-05-27,35576,PUT,2021-05-27,30657,CALL
9,15150,2021-05-27,9062,PUT,2021-05-27,6988,CALL


In [31]:
df['last_minute_OI_x'] = df['openInterest_x']
df['last_minute_OI_y'] = df['openInterest_y']
# df = df[['strikePrice','Time' ,'expiryDate_x', 'openInterest_x', 'last_minute_OI_x', 'RIGHT_x', 'openInterest_y', 'last_minute_OI_y', 'RIGHT_y']]
# df

# Keep this before

In [37]:
import time
for i in sp_range:
    d = datetime.now().time()
    df['Time'] = d.strftime('%H:%M')
    append_df_to_excel("master.xlsx",df,sheet_name=str(i),header=None, index=False)

KeyboardInterrupt: 

In [ ]:
# import pandas as pd
# import xlsxwriter

# df1 = pd.DataFrame({'Data': ['a', 'b', 'c', 'd']})

# df2 = pd.DataFrame({'Data': [1, 2, 3, 4]})

# df3 = pd.DataFrame({'Data': [1.1, 1.2, 1.3, 1.4]})

# writer = pd.ExcelWriter('multiple.xlsx', engine='xlsxwriter')

# df1.to_excel(writer, sheet_name='Sheet_1')

# df2.to_excel(writer, sheet_name='Sheet_2')

# df3.to_excel(writer, sheet_name='Sheet_3')

# writer.save()

# Initial Function - to be run once

In [96]:
from selenium import webdriver
import pandas as pd
import numpy as np
import time
import json

import requests

def main(time):

    while(True):
        headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15","Accept-Language": "en-gb","Accept-Encoding":"br, gzip, deflate","Accept":"test/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Referer":"http://www.google.com/"}
        url = "https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY"
        page = requests.get(url,headers=headers)

        if page.ok:
            break

    selected_margin = nifty_margin
   
    res = json.loads(page.text)['records']['data']
    underlying_value = int(res[0]['PE']['underlyingValue'])

    data = [d.get('CE') for d in res]
    data_cleaned = list(filter(None, data))
    df_call = pd.DataFrame(data_cleaned)

    data = [d.get('PE') for d in res]
    data_cleaned = list(filter(None, data))
    df_put = pd.DataFrame(data_cleaned)

    df_put['expiryDate'] = pd.to_datetime(df_put['expiryDate'])
    df_call['expiryDate'] = pd.to_datetime(df_call['expiryDate'])


    df_put.sort_values(by=['expiryDate','strikePrice'],inplace=True)
    df_call.sort_values(by=['expiryDate','strikePrice'],inplace=True)

    df_put = df_put[col_list]
    df_call = df_call[col_list]

    df_put = df_put[df_put['expiryDate'] == date]
    df_call = df_call[df_call['expiryDate'] == date]

    df_put['strikePrice'] = df_put['strikePrice'].map(int)
    df_call['strikePrice'] = df_call['strikePrice'].map(int)

    df_put = df_put[(df_put['strikePrice'] >= underlying_value-selected_margin) & (df_put['strikePrice']<=underlying_value+selected_margin)]
    df_call = df_call[(df_call['strikePrice'] >= underlying_value-selected_margin) & (df_call['strikePrice']<=underlying_value+selected_margin)]
    df_put['RIGHT'] = 'PUT'
    df_call['RIGHT'] = 'CALL'
    df_merged = df_put.merge(df_call, on='strikePrice')
    df_merged.insert(loc=0, column='Time', value=current_time)
    
#     print(df_merged)
    for i in sp_range:
        grouped = df.groupby(df.strikePrice)
        df_grouped = grouped.get_group(i)
#         print(df_grouped)
        append_df_to_excel("master.xlsx",df_grouped,sheet_name=str(i), header=None, index=False)
    
    
    

In [97]:
from datetime import datetime
import pandas as pd
import xlsxwriter

while(True):
        headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15","Accept-Language": "en-gb","Accept-Encoding":"br, gzip, deflate","Accept":"test/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Referer":"http://www.google.com/"}
        url = "https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY"
        page = requests.get(url,headers=headers)

        if page.ok:
            break

import json
res = json.loads(page.text)['records']['data']
underlying_value = int(res[0]['PE']['underlyingValue'])

data = [d.get('PE') for d in res]
data_cleaned = list(filter(None, data))
df_put = pd.DataFrame(data_cleaned)
df_put['expiryDate'] = pd.to_datetime(df_put['expiryDate'])
df_put.sort_values(by=['expiryDate','strikePrice'],inplace=True)

df_put = df_put[col_list]

df_put = df_put[df_put['expiryDate'] == date]

df_put['strikePrice'] = df_put['strikePrice'].map(int)

df_put = df_put[(df_put['strikePrice'] >= underlying_value-selected_margin) & (df_put['strikePrice']<=underlying_value+selected_margin)]

sp_range = df_put.strikePrice.unique().tolist()  

data_format = ['Time', 'Date','Stock/Index','Expiry','Right','Current OI','Previous Minute OI','OI Change',
               '% Up/Down from Last Minute OI','Right','Current OI','Previous Minute OI','OI Change',
               '% Up/Down from Last Minute OI']
base_df = pd.DataFrame(columns = data_format)
    
for i in sp_range:
    append_df_to_excel("master.xlsx",base_df,sheet_name=str(i), index=False)
    

while(True):
    now = datetime.now()
    current_time = now.strftime("%H:%M")
    print("Current Time =", current_time)
    main(current_time)
    time.sleep(60)

Current Time = 23:05
   strikePrice expiryDate_x  openInterest_x RIGHT_x expiryDate_y  \
0        14700   2021-05-27           31764     PUT   2021-05-27   

   openInterest_y RIGHT_y  
0           10115    CALL  
   strikePrice expiryDate_x  openInterest_x RIGHT_x expiryDate_y  \
1        14750   2021-05-27            7128     PUT   2021-05-27   

   openInterest_y RIGHT_y  
1             653    CALL  
   strikePrice expiryDate_x  openInterest_x RIGHT_x expiryDate_y  \
2        14800   2021-05-27           40094     PUT   2021-05-27   

   openInterest_y RIGHT_y  
2           22670    CALL  
   strikePrice expiryDate_x  openInterest_x RIGHT_x expiryDate_y  \
3        14850   2021-05-27            7821     PUT   2021-05-27   

   openInterest_y RIGHT_y  
3            1022    CALL  
   strikePrice expiryDate_x  openInterest_x RIGHT_x expiryDate_y  \
4        14900   2021-05-27           39429     PUT   2021-05-27   

   openInterest_y RIGHT_y  
4           13897    CALL  
   strikePrice

KeyboardInterrupt: 

# BAckup Codes for References

In [ ]:
# import pandas as pd
# import xlsxwriter

# df1 = pd.DataFrame({'Data': ['a', 'b', 'c', 'd']})

# df2 = pd.DataFrame({'Data': [1, 2, 3, 4]})

# df3 = pd.DataFrame({'Data': [1.1, 1.2, 1.3, 1.4]})

# writer = pd.ExcelWriter('multiple.xlsx', engine='xlsxwriter')

# df1.to_excel(writer, sheet_name='Sheet_1')

# df2.to_excel(writer, sheet_name='Sheet_2')

# df3.to_excel(writer, sheet_name='Sheet_3')

# writer.save()

In [ ]:
# import pandas as pd

# df1 = pd.DataFrame({'Data': ['a', 'b', 'c', 'd'], 'D': ['a', 'bb', 'c', 'dd']})

# df2 = pd.DataFrame({'Data': [1, 2, 3, 4],  'D': ['a', 'bb', 'c', 'dd']})

# c = pd.concat([df1, df2], axis=1)
# c